# Geocoding using an online API

We're going to explore geocoding using LocationIQ's API. If you've never used an API before this might be a fun introduction to how most servers talk to each other on the web.

Before you get going you will need to have a LocationIQ API key - something that authenticates you as 'not a bot' to LocationIQ's servers. You can get one by heading over here and signing up for an account: https://locationiq.com

Then you'll need to create an access token to let Python access the API. You can do this here: https://my.locationiq.com/dashboard#account

Once you've got your access token you should include it in place of the one below.

In [ ]:
import requests
from io import BytesIO
from cartopy import crs
from cartopy import feature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# If cartopy is not installed, you might need to try:
#!conda install cartopy

In [ ]:
YOUR_PRIVATE_TOKEN = 'pk.dcd2364cf53132c8e7f541906311234e' # put your own token in here...

You can continue to use this token to get running today (it's linked to my account) but I'll nuke the credentials after the class so this notebook will likely stop working after that. I'd suggest you make sure you're up and running with the API registration today if you want to use this code later.

We're going to use the `requests` library to send and recieve data from LocationIQ's API over the network. This is baked into every conda install so you don't have to install it yourself. For more on `requests` read [the documentation here](http://docs.python-requests.org/en/master/).

Let's take a look at what we can do with the API. Head over to LocationIQ's documentation page on 'forward geocoding' and browse around:

https://locationiq.com/docs#forward-geocoding

You'll notice that on the left there are a bunch of tabs to show example code in different languages. Click on the Python one and you'll get something like the following code:

In [ ]:
# The URL that we're going to hit
url = "https://us1.locationiq.com/v1/search.php"
data = {
    'key': YOUR_PRIVATE_TOKEN,     # our token from above
    'q': 'Empire State Building',  # the place that we want to search for
    'format': 'json'               # how we want to get the results
}

# Actually hit the endpoint
response = requests.get(url, params=data)

What this does is creates an HTTP request object, hit the endpoint over the web and then returns the JSON document that it gets.

We can look at individual entries such as the URL:

In [ ]:
response.url

and whether the response returned normally

In [ ]:
response.ok

...which is quite nice as you can do something like the following:

```python
response = requests.get('http://some.url')
if response.ok:
    data = response.json()
else:
    raise ValueError("Couldn't hit URL!")  # please provide a better error message than this!
```

The other nice thing that requests does is automatically parse the returned text into a Python dictionary for us. Let's take a look at that:

In [ ]:
response.json()

---

___Aside: getting more than JSON from an endpoint___

_If you want to just get the raw binary string from the endpoint then you can do so with the `content` attribute. This is useful if you're getting images back (say a `.png`) and want to pass them to scipy to read into an image._

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/f/f4/Perth-skyline.jpg'

picture = requests.get(url)
if picture.ok:
    image = plt.imread(BytesIO(picture.content),'jpg')
    plt.imshow(image)


_For example we can hit the icons returned in the API if we wanted to render these_

In [ ]:
result = response.json()[0]
result['icon']  # has an URL pointing to an icon for us to render if we want

In [ ]:
def get_picture(url,ext='png'):
    """ Get a picture from some URL 
    
    """
    picture = requests.get(url)
    if picture.ok:
        return plt.imread(BytesIO(picture.content),ext)
    else:
        raise ValueError('Something went wrong!')

In [ ]:
# Loop through our responses and plot the icon in a seperate axis
fig, axes = plt.subplots(1, len(response.json()))
for result, ax in zip(response.json(), axes):
    icon = get_picture(result['icon'])
    ax.imshow(icon)
    ax.set_axis_off()

_Now you'd have to work out how to put those in the right location on the map - at this point some searching and StackOverflow would probably be helpful_

---

Ok back to our original data from the API. So it turns out that there are several 'places' associated with the name 'Empire State Building'. In particular we now have point locations (latitude and longitude) and even a full display address that we could use to add value to our data. Which one you pick is up to you but note that LocationIQ also provides an 'importance' attribute that you could use to filter your results.

For now, let's extract the latitude and longitude and put this on a map, all in Python. We're going to use `cartopy` to generate our maps and handle the projections for us. For more on `cartopy` check out [its documentation here](https://scitools.org.uk/cartopy/docs/latest/index.html). 

The first time you run this you might get a warning that cartopy is downloading the Natural Earth coastline data. You can read more about the Natural Earth datasets here: https://www.naturalearthdata.com. This is a 

In [ ]:
ax = plt.axes(projection=crs.Orthographic(
    central_latitude=65,
    central_longitude=-50
    
))
ax.coastlines()

You can have other basemaps or projections if you want as well

In [ ]:
ax = plt.axes(projection=crs.AlbersEqualArea(
    central_latitude=65,
    central_longitude=-50
    
))
ax.stock_img()
ax.set_extent((-90, 40, -60, 60))

If you want a different projection, try one of the others from the `cartopy` project list here: https://scitools.org.uk/cartopy/docs/v0.9/crs/projections.html

In [ ]:
ax = plt.axes(projection=crs.PlateCarree())
ax.stock_img()

So now we can parse the latitude and longitude points from the API and plot these

In [ ]:
# Extract our points for each hit
points = np.array([
    (float(hit['lon']), float(hit['lat'])) for hit in response.json()
]).transpose()

# Generate our basemap & points
proj = crs.PlateCarree()
ax = plt.axes(projection=proj)
ax.set_extent((-90, 40, 20, 60))  # sets bounds as (min_lon, max_lon, min_lat, max_lat)
plt.plot(points[0], points[1], 'ro')
ax.stock_img()
ax.coastlines()

# Add states and provinces from Natural Earth
# see https://www.naturalearthdata.com/features/ for more things to plot
countries = feature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')
ax.add_feature(countries, edgecolor='gray')

# Resize the image to make it a bit bigger
plt.gcf().set_size_inches(14, 8)

Congratulations, you've made a Python version of Google Maps (without the UI polish of the original)!

If you want to make a pointy-clicky-scrolly version then take a look at folium https://github.com/python-visualization/folium, although if you're doing this seriously then you probably want to switch to a better front end language than Python (i.e. JavaScript). Typically geospatial processing code uses Python in the backend and JS for rendering in the frontend.

## Exercise

1. Write a function `get_position` that geocodes a given location. Start with the following

```python
def get_position(location_string):
    """ Get the latitude and longitude of the given location
    
        Returns the location with the highest likelihood from the LocationIQ API
     
        Parameters:
            location_string - The location to geocode, for example
                'Eiffel Tower' or 'Empire State Building'
        
        Returns:
            the location as a numpy array: [longitude, latitude]
    """
    # your code here
```

2. Then make a list of addresses in Pandas (as a Pandas Series) and apply your function to generate longitude & latitude columns in the data

#### Note: If not using a personal token and on Binder
There may be response issues where a more robust approach to using the response package might be needed, see https://realpython.com/python-requests/#performance

#### _Bonus: Plot the locations a map!_

In [ ]:
def get_position(location_string):
    """ Get the latitude and longitude of the given location
    
        Returns the location with the highest likelihood from the LocationIQ API
     
        Parameters:
            location_string - The location to geocode, for example
                'Eiffel Tower' or 'Empire State Building'
        
        Returns:
            the location as a numpy array: [longitude, latitude]
    """
    # Talk to the API 
    token = 
    url =
    data = {
        'key': ,
        'q': ,
        'format': 'json'
    }
    response = 
    
    # Check results
    if not response.ok:
        raise ValueError('Response not ok')
    
    # Get best result
    results = [
        (float(result['lat']), float(result['lon']), result['importance']) 
        for result in response.json()
    ]
    best_result = sorted(results, key=lambda x: x[2])[-1]
    lat, long, _ = best_result
    return np.array([lat, long])


In [ ]:
position = get_position('Eiffel tower')
position

In [ ]:
test_data = pd.Series([
    
])